In [ ]:
# ONS benefits claimants for Lewisham.
# Loads ONS data (from Nomis) and prepares it for use in Lewisham ward profiles.
#
# This requires the LSOA->WD22 lookup table produced by "lbl_boundaries_2022_wards.ipynb"

In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Tools
==
Helpers to reformat & aggregate the source data.

In [2]:
# Aggregate OA-level data to LSOAs, Wards, etc. (Calculates the sum.)
def aggregate_oa_groups(oa_data, oa_group_table, oa_col, group_col):
  return pd.merge(oa_group_table, oa_data, on=oa_col, how='right').\
              drop(columns=[oa_col]).\
              groupby(group_col).sum()

Data
==
GDrive mount
--

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
out_dir = f"{project_dir}/benefits"

In [ ]:
!mkdir -p '{out_dir}'

Downloads
--

In [ ]:
# TODO: use their API to get the data.
# Source: https://www.nomisweb.co.uk/datasets/ucjsa

# For now we manually downloaded some spreadsheets with their query builder.
# Details here: https://github.com/lb-lewisham/lewisham-ward-profiles/issues/62

In [ ]:
!ls -lh "{out_dir}"

total 168K
-rw------- 1 root root 111K Nov  3 16:17 nomis_2021_11_03_161616.xlsx
-rw------- 1 root root  24K Nov  3 16:30 nomis_2021_11_03_162932.xlsx
-rw------- 1 root root  34K Nov  3 16:36 nomis_2021_11_03_163457.xlsx


Lookups
--
Used to match up LSOAs with 2022 electoral wards

In [5]:
# All LSOAs and their 2022 wards in Lewisham
lsoa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_lsoa11_wd22_proposed.csv")
lsoa_wd22_join = lsoa_wd22_join[['LSOA11CD', 'WD22CD_proposed']]
lsoa_wd22_join.head()

,LSOA11CD,WD22CD_proposed
0,E01003316,E05013729
1,E01003314,E05013729
2,E01003312,E05013729
3,E01003313,E05013729
4,E01003310,E05013729


Process & export
==

In [6]:
# For filtering of source data
lbl_lsoa_list = lsoa_wd22_join.LSOA11CD.unique() # All LSOAs in Lewisham
len(lbl_lsoa_list)

169

Time series by gender
--

In [ ]:
# d = pd.read_excel(f"{out_dir}/nomis_2021_11_03_161616.xlsx", 
#                   sheet_name=0, # First sheet: totals
#                   skiprows=7, # Data starts in row 8
#                   skipfooter=5) # Data has some footer rows
# d.tail()

In [ ]:
# d.columns

In [ ]:
# Raw single-year plus totals
for sheetname, datname in zip(
    [0, 1, 2],
    ['benefits_claimants_total', 'benefits_claimants_male', 'benefits_claimants_female']):

  IPython.display.display(f"== {sheetname} ==")

  d = pd.read_excel(f"{out_dir}/nomis_2021_11_03_161616.xlsx", 
                    sheet_name=sheetname, # First sheet: totals
                    skiprows=7, # Data starts in row 8
                    skipfooter=5) # Data has some footer rows
  d = d.rename(columns={'Unnamed: 1': 'LSOA11CD'}) # The geography column is unnamed in this data set
  #IPython.display.display(len(d))
  #IPython.display.display(d.head())

  # Filter & transform to LSOA index data
  lbl_lsoa = d[d.LSOA11CD.isin(lbl_lsoa_list)]#.drop(columns=['LSOA11CD'])
  lbl_lsoa.to_csv(f"{out_dir}/lbl_{datname}_lsoa11.csv", index=True)
  IPython.display.display(len(lbl_lsoa))
  IPython.display.display(lbl_lsoa.head())

  # Aggregate to WD22 level
  lbl_wd22 = aggregate_oa_groups(lbl_lsoa, lsoa_wd22_join, oa_col='LSOA11CD', group_col='WD22CD_proposed')
  lbl_wd22.to_csv(f"{out_dir}/lbl_{datname}_wd22.csv", index=True)
  IPython.display.display(len(lbl_wd22))
  IPython.display.display(lbl_wd22.head())

'== 0 =='

169

,2011 super output area - lower layer,LSOA11CD,January 2016,February 2016,March 2016,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017,February 2017,March 2017,April 2017,May 2017,June 2017,July 2017,August 2017,September 2017,October 2017,November 2017,December 2017,January 2018,February 2018,March 2018,April 2018,May 2018,June 2018,July 2018,August 2018,September 2018,October 2018,November 2018,December 2018,January 2019,February 2019,March 2019,April 2019,May 2019,June 2019,July 2019,August 2019,September 2019,October 2019,November 2019,December 2019,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020,January 2021,February 2021,March 2021,April 2021,May 2021,June 2021,July 2021,August 2021,September 2021
0,Lewisham 001A,E01003244,40,40,45,40,35,35,35,35,30,35,40,40,40,40,40,45,50,45,50,45,45,50,45,45,40,40,40,40,45,45,40,40,40,45,50,55,50,55,55,60,55,60,60,60,60,60,60,55,60,55,60,85,130,130,130,130,140,140,145,140,145,155,155,160,150,140,125,120,115
1,Lewisham 001B,E01003245,35,40,40,45,40,45,55,55,50,45,55,60,55,60,60,65,60,60,65,65,65,70,65,70,65,75,60,75,75,75,75,70,70,70,65,70,70,75,70,70,75,75,75,75,80,70,80,75,70,65,70,110,125,140,140,145,145,135,155,155,145,155,145,155,145,130,135,130,115
2,Lewisham 001C,E01003247,15,20,20,20,20,25,35,35,30,30,35,35,30,25,25,30,30,35,25,30,35,35,30,30,35,35,35,40,40,35,40,40,40,40,35,40,40,40,40,40,35,30,35,30,30,35,30,25,25,25,30,70,115,110,110,115,130,130,130,130,130,135,140,135,130,110,100,105,95
3,Lewisham 001D,E01003249,40,40,35,40,40,45,40,35,35,30,30,30,30,30,35,45,45,40,45,40,45,45,40,40,45,40,40,35,40,40,40,40,45,50,50,50,55,65,65,65,65,60,60,60,55,55,60,50,50,50,50,95,130,130,140,140,145,155,155,155,155,165,175,165,160,145,140,125,120
4,Lewisham 001E,E01003250,50,55,55,50,55,45,45,50,50,45,45,45,50,50,45,50,50,50,50,55,50,50,50,50,50,50,55,55,55,60,60,55,55,55,50,55,60,55,60,65,60,60,65,65,60,65,65,65,60,55,55,95,150,155,155,155,165,165,165,170,180,185,185,185,185,170,170,160,150


19

,January 2016,February 2016,March 2016,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017,February 2017,March 2017,April 2017,May 2017,June 2017,July 2017,August 2017,September 2017,October 2017,November 2017,December 2017,January 2018,February 2018,March 2018,April 2018,May 2018,June 2018,July 2018,August 2018,September 2018,October 2018,November 2018,December 2018,January 2019,February 2019,March 2019,April 2019,May 2019,June 2019,July 2019,August 2019,September 2019,October 2019,November 2019,December 2019,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020,January 2021,February 2021,March 2021,April 2021,May 2021,June 2021,July 2021,August 2021,September 2021
WD22CD_proposed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E05013714,255,275,260,295,280,275,275,265,245,255,245,220,235,265,285,305,305,305,305,290,290,295,285,290,315,335,335,325,310,325,300,280,290,310,325,360,355,365,375,385,395,405,410,425,435,440,450,465,455,480,495,630,910,910,925,945,955,950,945,945,935,990,1000,985,955,885,860,850,825
E05013715,235,245,245,255,250,255,255,275,275,255,255,250,260,255,275,280,270,280,260,270,270,270,270,255,255,240,235,255,235,240,215,225,230,235,240,255,265,270,305,300,305,320,335,335,350,345,345,345,350,370,385,610,875,840,850,855,880,875,910,910,920,915,940,945,870,830,795,760,750
E05013716,360,370,370,380,370,380,360,355,360,350,340,345,365,350,375,385,400,400,385,390,395,390,380,380,390,390,395,405,395,395,390,385,425,435,450,460,465,485,510,500,505,540,550,540,555,545,540,535,530,565,550,925,1445,1400,1465,1570,1580,1525,1555,1555,1525,1565,1605,1555,1485,1375,1320,1265,1235
E05013717,260,255,255,250,235,210,235,245,255,255,240,235,245,250,250,270,265,255,270,240,250,245,235,230,240,250,250,260,245,245,225,215,215,235,240,275,285,290,305,315,320,330,335,325,355,350,360,375,390,405,410,615,965,935,970,990,1025,1005,1025,1020,1005,1045,1065,1080,1025,945,900,865,820
E05013718,220,225,235,240,240,220,220,215,200,205,190,180,200,210,220,205,185,190,190,175,175,195,205,180,190,195,200,215,215,200,190,185,205,215,215,215,235,255,265,270,265,280,290,305,300,320,305,305,305,305,290,540,830,825,820,865,860,815,850,840,810,860,875,835,805,730,690,655,645


...

'== 2 =='

169

,2011 super output area - lower layer,LSOA11CD,January 2016,February 2016,March 2016,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017,February 2017,March 2017,April 2017,May 2017,June 2017,July 2017,August 2017,September 2017,October 2017,November 2017,December 2017,January 2018,February 2018,March 2018,April 2018,May 2018,June 2018,July 2018,August 2018,September 2018,October 2018,November 2018,December 2018,January 2019,February 2019,March 2019,April 2019,May 2019,June 2019,July 2019,August 2019,September 2019,October 2019,November 2019,December 2019,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020,January 2021,February 2021,March 2021,April 2021,May 2021,June 2021,July 2021,August 2021,September 2021
0,Lewisham 001A,E01003244,25,25,30,30,20,20,20,15,15,15,20,20,15,20,20,20,20,20,25,20,20,25,25,20,20,20,20,20,20,20,20,20,20,25,25,30,25,25,25,30,25,30,30,30,30,30,30,25,30,30,30,40,55,55,60,50,55,60,65,60,65,70,70,70,70,60,50,50,50
1,Lewisham 001B,E01003245,15,15,15,15,10,15,20,20,20,15,25,30,30,25,25,30,30,30,35,35,35,40,35,40,35,40,30,35,35,40,35,35,35,35,35,40,40,40,40,40,45,50,45,45,45,40,45,40,40,35,40,65,65,75,75,75,75,65,80,70,70,75,75,75,70,60,60,55,50
2,Lewisham 001C,E01003247,5,5,5,5,5,10,10,10,10,10,10,10,10,10,5,10,10,15,15,15,20,20,15,15,15,15,20,20,20,20,20,20,20,15,15,20,15,15,15,20,15,10,15,15,15,15,15,10,10,10,10,30,40,45,45,50,60,60,55,55,55,55,60,60,50,40,45,40,35
3,Lewisham 001D,E01003249,10,10,10,10,10,15,10,5,10,5,10,5,10,10,10,15,15,10,10,10,15,15,10,15,10,10,10,10,10,10,10,10,10,15,15,20,20,20,25,25,25,20,20,20,15,15,20,15,15,15,20,35,45,45,50,45,50,55,50,45,50,55,55,50,50,50,50,55,50
4,Lewisham 001E,E01003250,15,15,15,15,15,10,15,15,15,15,15,15,15,15,15,15,15,15,20,20,20,20,20,20,20,20,20,25,25,25,25,20,25,25,20,25,25,20,20,25,25,25,30,30,30,35,30,30,25,30,30,40,55,60,55,65,60,65,70,75,75,80,80,85,80,70,75,70,65


19

,January 2016,February 2016,March 2016,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017,February 2017,March 2017,April 2017,May 2017,June 2017,July 2017,August 2017,September 2017,October 2017,November 2017,December 2017,January 2018,February 2018,March 2018,April 2018,May 2018,June 2018,July 2018,August 2018,September 2018,October 2018,November 2018,December 2018,January 2019,February 2019,March 2019,April 2019,May 2019,June 2019,July 2019,August 2019,September 2019,October 2019,November 2019,December 2019,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020,January 2021,February 2021,March 2021,April 2021,May 2021,June 2021,July 2021,August 2021,September 2021
WD22CD_proposed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E05013714,105,105,110,120,115,120,110,110,100,105,90,75,80,95,105,110,105,125,130,115,120,115,115,115,120,125,120,115,110,105,100,95,95,115,120,135,135,140,130,140,140,145,150,155,165,170,185,170,170,175,170,230,310,320,330,335,340,350,355,345,355,370,380,370,360,330,320,320,310
E05013715,95,105,105,105,105,115,115,110,120,115,105,100,100,105,110,115,120,125,125,125,125,120,115,90,120,110,110,115,100,110,110,110,105,100,110,120,115,115,125,135,140,140,130,145,140,150,155,155,155,160,175,270,385,380,375,390,410,390,430,400,410,405,420,430,390,365,370,360,350
E05013716,160,160,150,165,160,150,150,150,155,135,130,125,130,140,150,165,160,170,160,165,165,160,140,145,165,170,160,165,170,175,170,165,195,205,200,195,200,205,215,205,215,240,255,245,255,245,235,225,235,240,230,410,630,605,650,705,715,695,705,710,695,710,720,685,680,610,595,540,545
E05013717,85,100,90,95,90,85,110,125,120,120,115,115,105,115,120,115,115,105,110,100,100,100,85,90,90,90,90,95,90,95,85,80,95,105,105,120,120,125,150,135,140,150,145,160,170,150,150,160,165,175,175,275,405,410,425,435,450,420,445,445,445,455,460,455,440,405,365,360,335
E05013718,90,95,100,110,100,95,95,90,90,90,75,75,90,105,95,85,75,80,80,70,75,85,90,75,65,75,80,90,95,80,75,75,95,105,100,95,95,115,105,130,135,130,140,145,135,150,145,135,140,145,125,245,370,375,375,375,380,350,370,370,345,375,370,365,345,305,300,270,275


Results
--

In [ ]:
!ls -lh '{out_dir}'

total 307K
-rw------- 1 root root  39K Nov  3 17:24 lbl_benefits_claimants_female_lsoa11.csv
-rw------- 1 root root 6.0K Nov  3 17:24 lbl_benefits_claimants_female_wd22.csv
-rw------- 1 root root  40K Nov  3 17:24 lbl_benefits_claimants_male_lsoa11.csv
-rw------- 1 root root 6.1K Nov  3 17:24 lbl_benefits_claimants_male_wd22.csv
-rw------- 1 root root  42K Nov  3 17:24 lbl_benefits_claimants_total_lsoa11.csv
-rw------- 1 root root 6.3K Nov  3 17:24 lbl_benefits_claimants_total_wd22.csv
-rw------- 1 root root 111K Nov  3 16:17 nomis_2021_11_03_161616.xlsx
-rw------- 1 root root  24K Nov  3 16:30 nomis_2021_11_03_162932.xlsx
-rw------- 1 root root  34K Nov  3 16:36 nomis_2021_11_03_163457.xlsx


In [ ]:
# !rm '{out_dir}'/*.csv